<center><u><b><h1 style="color:Red;">Titanic : Machine Learning from Disaster</u></b></h1></center>

In [110]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load data
train_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\train.csv")
test_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")

# Combine train and test datasets to handle them together
combine = [train_df, test_df]

# Feature Engineering
for dataset in combine:
    # Create new feature FamilySize
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    # Create new feature IsAlone
    dataset['IsAlone'] = 1
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0
    # Extract titles from names
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# Fill missing values
for dataset in combine:
    dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace=True)

# Combine datasets for preprocessing
test_df['Survived'] = np.nan  # Add a dummy 'Survived' column to align columns
combined_df = pd.concat([train_df, test_df])

# Feature Engineering
combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
combined_df['IsAlone'] = 1
combined_df['IsAlone'].loc[combined_df['FamilySize'] > 1] = 0
combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
combined_df['Title'] = combined_df['Title'].replace('Mlle', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Ms', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Mme', 'Mrs')

# Fill missing values
combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)
combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)

# Convert categorical features to numeric
combined_df['Sex'] = combined_df['Sex'].map({'female': 1, 'male': 0}).astype(int)
combined_df['Title'] = combined_df['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}).astype(int)
combined_df['Embarked'] = combined_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

# Drop unnecessary features
combined_df = combined_df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)

# Separate back into train and test sets
train_df = combined_df[combined_df['Survived'].notna()]
test_df = combined_df[combined_df['Survived'].isna()].drop('Survived', axis=1)

X_train_full = train_df.drop('Survived', axis=1)
y_train_full = train_df['Survived']
X_test = test_df.copy()

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.12, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Build the model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Detailed classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

# Make predictions on the test set
predictions = model.predict(X_test).astype(int)

# Create submission file
submission = pd.DataFrame({
    "PassengerId": pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")["PassengerId"],
    "Survived": predictions
})

submission.to_csv(r"C:\Kaggles\Titanic\Submissions\Submission_2.csv", index=False)


Validation Accuracy: 0.8411214953271028
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.88      0.87        65
         1.0       0.80      0.79      0.80        42

    accuracy                           0.84       107
   macro avg       0.83      0.83      0.83       107
weighted avg       0.84      0.84      0.84       107

Confusion Matrix:
[[57  8]
 [ 9 33]]


C:\Users\asus\AppData\Local\Temp\ipykernel_19188\3452088656.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0
C:\Users\asus\AppData\Local\Temp\ipykernel_19188\3452088656.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0
C:\Users\asus\AppData\Local\Temp\ipykernel_19188\3452088656.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a